# 情感分类部分

In [1]:
# 安装必要的库
# !pip install mecab-python3
# !pip install transformers
# !pip install pandas
import MeCab
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

def main1(text1):
    def preprocess_text(text):
        return mecab.parse(text).strip()
    # 预处理文本
    preprocessed_text = preprocess_text(text1)

    # 执行情感分析
    result = sentiment_analyzer(preprocessed_text)
    label = result[0]['label']
    score = result[0]['score']
    return label,score


if __name__ == '__main__':
    # 初始化 MeCab 进行分词
    mecab = MeCab.Tagger("-Owakati")
    # 使用 transformers 中的预训练情感分析模型
    sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
    df = pd.read_excel('Twitter_data1.xlsx')
    data = pd.DataFrame()
    data['推文内容'] = ['推文内容']
    data['关键词'] = ['关键词']
    data['发布时间'] = ['发布时间']
    data['label'] =['label']
    data['score'] = ['score']
    data.to_csv('data.csv', encoding='utf-8-sig', index=False, mode='w',header=False)
    for d1,d2,d3 in tqdm(zip(df['推文内容'],df['关键词'],df['发布时间'])):
        text = d1
        label,score = main1(text)
        data = pd.DataFrame()
        data['推文内容'] = [text]
        data['关键词'] = [d2]
        data['发布时间'] = [d3]
        data['label'] = [label]
        data['score'] = [score]
        data.to_csv('data.csv',encoding='utf-8-sig',index=False,mode='a+',header=False)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
10it [00:02,  4.34it/s]


In [11]:
import pandas as pd

df = pd.read_excel('丰田股价20230531.xlsx')
print(df['日付'])
# 计算环比（当前值与前一个值的差异）
df['环比'] = df['終値'].pct_change()

# 判断环比结果，正数为1，负数或0为0
df['结果'] = df['环比'].apply(lambda x: 1 if x > 0 else 0)
df['日付'] = pd.to_datetime(df['日付'])
df.to_excel('股票.xlsx')

0     2024-05-31
1     2024-05-30
2     2024-05-29
3     2024-05-28
4     2024-05-27
         ...    
241   2023-06-06
242   2023-06-05
243   2023-06-02
244   2023-06-01
245   2023-05-31
Name: 日付, Length: 246, dtype: datetime64[ns]


In [12]:
import pandas as pd
import numpy as np

data1 = pd.read_csv('data1.csv')
data2 = pd.read_csv('data2.csv')

df = pd.concat([data1,data2],axis=0)
df = df.drop_duplicates(subset=['推文内容'])

def pandaun(s):
    s1 = float(s)
    if s1 >= 0.7:
        return s1
    else:
        return np.NAN



def time_process(x):
    x1 = str(x).split(" ")
    return x1[0]


df['时间'] = df['发布时间'].apply(time_process)
df['发帖数量'] = 1
df['准确率'] = df['score'].apply(pandaun)
df1 = df.dropna(subset=['准确率'],axis=0)
def df_process(data):
    data1 = data['label'].value_counts()
    d = {}
    for i,j in zip(data1.index,data1.values):
        d[i] = j
    return d

df11 = df1.groupby(by=['时间']).apply(df_process)
df3 = df.groupby(by=['时间']).agg({'发帖数量':'sum'})
df11 = df11.reset_index()
df11.columns = ['时间','类别']
df11 = df11.sort_index()

df4 = df3.reset_index()
df4.columns = ['时间','发帖数量']
df4 = df4.sort_index()

df5 = pd.merge(df11,df4,on='时间')

df2 = df5


list_0 = []
list_1 = []
list_2 = []
for d in df2['类别']:
    try:
        LABEL_0 = d['LABEL_0']
    except:
        LABEL_0 = 0
    try:
        LABEL_1 = d['LABEL_1']
    except:
        LABEL_1 = 0
    try:
        LABEL_2 = d['LABEL_2']
    except:
        LABEL_2 = 0
    list_0.append(LABEL_0)
    list_1.append(LABEL_1)
    list_2.append(LABEL_2)


df2['消极'] = list_0
df2['中立'] = list_1
df2['积极'] = list_2
df2 = df2.drop(['类别'],axis=1)
data = pd.read_excel('股票.xlsx')
df2['时间'] = pd.to_datetime(df2['时间'])
data['时间'] = pd.to_datetime(data['日付'])
data1 = pd.merge(df2,data,on='时间',how='right')
data1 = data1.dropna(how='any',axis=0)

data1.to_excel('情感分类数据.xlsx',index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2668\3968472636.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df11 = df1.groupby(by=['时间']).apply(df_process)
